# Summarize K-Means (K=1000) Cluster for SVD

In [1]:
# modify these for your own computer
repo_directory = '/Users/Michael/Documents/GitHub/law-net/'

data_dir = '/Users/Michael/Desktop/network_data/'

import os
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
import glob
import cPickle  as pickle
from collections import OrderedDict
import json


# graph package
import igraph as ig


# stat
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_mutual_info_score as ami
from sklearn.metrics import mutual_info_score as mi
from sklearn.metrics import adjusted_rand_score as ar
from sklearn.metrics import calinski_harabaz_score as ch # (X, labels)
from sklearn.metrics import completeness_score as cs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import fowlkes_mallows_score as fm
from sklearn.metrics import homogeneity_completeness_v_measure as hcvm
from sklearn.metrics import homogeneity_score as hs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import silhouette_score as ss # (X, labels)
from sklearn.metrics import silhouette_samples as ss2 # (X, labels)
from sklearn.metrics import v_measure_score as vm

import scipy.sparse
import random
import itertools
from itertools import combinations


# our code
sys.path.append(repo_directory + 'code/')
from summarize_clusters import *
from helpful_functions import *

sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
from bag_of_words import * 

# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'
nlp_sub_dir = nlp_dir + 'bow_tfidf/' #tfidf matrix (and other info, i.e. vocab) computed from bag-of-words matrix
nlp_bow_dir = nlp_dir + 'bow/' #bag-of-words matrix (and other info, i.e. vocab)
nlp_df_sub_dir = nlp_dir + 'bow_tfidf_df/'

# csv location
csv_dir = "C:/Users/Michael/Documents/GitHub/law-net/csv/"
csv_dir_SVD_km1000 = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_SVD_km1000/"
csv_dir_SVD_km1000_info = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_SVD_km1000/info/"
csv_dir_SVD_km1000_summary = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_SVD_km1000/summary/"

# all the file paths for .txt files
file_paths = glob.glob(text_dir + '*.txt')

# all opinions
all_the_opinions = all_opinions(file_paths)

# clusters directory
clusters_dir = "C:/Users/Michael/Desktop/network_data/raw/scotus/clusters/"

# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

## load tf-idf vectors
**tfidf_matrix** = (row_index, column_index): tf_idf value (**CSR FORMAT**)  
**op_id_to_bow_id** = opinion_id (corresponds to row indices)  
**vocab** = all the words in tfidf_matrix (correspond to column indices)

In [2]:
tfidf_matrix, op_id_to_bow_id, vocab = load_tf_idf(nlp_sub_dir)

In [3]:
tfidf_matrix

<27885x567570 sparse matrix of type '<type 'numpy.float64'>'
	with 20817470 stored elements in Compressed Sparse Row format>

In [4]:
clusters = pd.read_csv(csv_dir + 'clusters_SVD.csv')
clusters.head()

,Unnamed: 0,km_10,km_100,km_1000,gmm_10,gmm_100,gmm_1000,hc_10,hc_100,hc_1000
0,145658,0,1,224,3,54,392,1,50,157
1,89370,0,68,569,3,95,830,1,50,157
2,89371,9,85,886,8,47,711,2,94,697
3,89372,0,70,242,3,40,443,1,37,193
4,89373,0,22,897,0,64,213,1,65,84


In [5]:
nlp_tfidf_clusters = clusters['km_1000'].tolist()

nlp_clusters = pd.Series(nlp_tfidf_clusters, index=op_id_to_bow_id)
nlp_clusters.head()

145658    224
89370     569
89371     886
89372     242
89373     897
dtype: int64

## get the top 5 biggest clusters

In [6]:
'''
dict_top_n_clusters = dictionary of top K clusters 
                      (key=cluster #, value=opinions in cluster)
                      
biggest_n_clusters = list of top K clusters (int)
'''

dict_top_n_clusters, biggest_n_clusters = get_top_n_clusters(1000, 1000, nlp_clusters)

cluster 242 : 237 opinions
cluster 425 : 182 opinions
cluster 764 : 176 opinions
cluster 609 : 174 opinions
cluster 390 : 162 opinions
cluster 709 : 160 opinions
cluster 3 : 147 opinions
cluster 451 : 143 opinions
cluster 688 : 113 opinions
cluster 232 : 92 opinions
cluster 351 : 92 opinions
cluster 791 : 91 opinions
cluster 319 : 90 opinions
cluster 634 : 89 opinions
cluster 832 : 89 opinions
cluster 521 : 88 opinions
cluster 800 : 86 opinions
cluster 844 : 84 opinions
cluster 11 : 83 opinions
cluster 424 : 83 opinions
cluster 185 : 81 opinions
cluster 166 : 80 opinions
cluster 340 : 80 opinions
cluster 483 : 80 opinions
cluster 767 : 79 opinions
cluster 67 : 77 opinions
cluster 464 : 77 opinions
cluster 229 : 72 opinions
cluster 230 : 72 opinions
cluster 525 : 71 opinions
cluster 65 : 68 opinions
cluster 81 : 68 opinions
cluster 99 : 67 opinions
cluster 801 : 67 opinions
cluster 868 : 67 opinions
cluster 897 : 67 opinions
cluster 162 : 66 opinions
cluster 388 : 65 opinions
cluster 61

# Top K Words of Each Cluster
This function summarizes a set of opinions by returning the words that appear in these opinions with the highest tf-idf scores.

# Top K Words ($\mu_{cluster}$) of Each Cluster
compute the mean tf-idf vector of the cluster, return the top K words from this mean vector

# Top K Words ($\mu_{cluster} - \mu_{complement}$ ) of Each Cluster
compute the mean tf-idf vector of the cluster and also of the complement of the cluster,  
take the difference mu_cluster - mu_complement, return the top K words in this difference

# Most Relevant Opinion of Each Cluster
compute the mean tf-idf vector, return the document in the cluster closet to the mean  

In [7]:
def cluster_infos_csv(biggest_n_clusters, dict_top_n_clusters, clusters_dir, csv_dir_info):
    
    for i in biggest_n_clusters:
        
        opinion_names = []
        opinion_dates = []
        opinion_links = []
        
        for j in dict_top_n_clusters[i]:
            try:
                with open(clusters_dir + j + ".json") as data_file:
                    data = json.load(data_file)
            except IOError:
                pass
                #name, date, link = case_info2(i)
                #opinion_names.append(name)
                #opinion_dates.append(date)
                #opinion_links.append(link)
            
            name = data['case_name'].encode('utf-8')
            date = data['date_filed'].encode('utf-8')
            link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')

            opinion_names.append(name)
            opinion_dates.append(date)
            opinion_links.append(link)

        cluster_info = pd.DataFrame()
        cluster_info['names'] = opinion_names
        cluster_info['dates'] = opinion_dates
        cluster_info['url'] = opinion_links

        cluster_info.to_csv(csv_dir_info + "cluster_"+str(i)+".csv")

In [8]:
%%time
cluster_infos_csv(biggest_n_clusters, dict_top_n_clusters, clusters_dir, csv_dir_SVD_km1000_info)

Wall time: 24.3 s


In [9]:
def cluster_summaries_csv(k, biggest_n_clusters, dict_top_n_clusters, tfidf_matrix, op_id_to_bow_id, vocab, csv_dir_summary):
    for i in biggest_n_clusters:
        top_words = top_k_words(dict_top_n_clusters[i], k, tfidf_matrix, op_id_to_bow_id, vocab)
        top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[i], k, tfidf_matrix, op_id_to_bow_id, vocab)
        top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[i], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
        most_relev_op = document_closest_to_mean(dict_top_n_clusters[i], tfidf_matrix, op_id_to_bow_id)
        
        top_words = [x.encode('utf-8') for x in top_words]
        top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
        top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

        len_old_top_words = len(top_words)
        len_old_top_words_from_mean = len(top_words_from_mean)
        len_old_top_words_from_diff = len(top_words_from_diff)

        if len(top_words) != len(top_words_from_mean):
            for j in range(0,len(top_words_from_mean)-len(top_words)):
                top_words.append(np.nan)

        cluster_summary = pd.DataFrame()
        cluster_summary['top_words'] = top_words
        cluster_summary['top_words_from_mean'] = top_words_from_mean
        cluster_summary['top_words_from_diff'] = top_words_from_diff
        cluster_summary['most_relev_op'] = most_relev_op

        cluster_summary.to_csv(csv_dir_summary + "cluster_"+str(i)+"_summary.csv")
        print "cluster", i, "is done", "(", len(dict_top_n_clusters[i]), "opinions )", len_old_top_words, len_old_top_words_from_mean, len_old_top_words_from_diff

In [10]:
%%time
cluster_summaries_csv(1000, biggest_n_clusters, dict_top_n_clusters, tfidf_matrix, op_id_to_bow_id, vocab, csv_dir_SVD_km1000_summary)

cluster 242 is done ( 237 opinions ) 1000 1000 1000
cluster 425 is done ( 182 opinions ) 1000 1000 1000
cluster 764 is done ( 176 opinions ) 1000 1000 1000
cluster 609 is done ( 174 opinions ) 1000 1000 1000
cluster 390 is done ( 162 opinions ) 1000 1000 1000
cluster 709 is done ( 160 opinions ) 1000 1000 1000
cluster 3 is done ( 147 opinions ) 1000 1000 1000
cluster 451 is done ( 143 opinions ) 1000 1000 1000
cluster 688 is done ( 113 opinions ) 1000 1000 1000
cluster 232 is done ( 92 opinions ) 1000 1000 1000
cluster 351 is done ( 92 opinions ) 1000 1000 1000
cluster 791 is done ( 91 opinions ) 1000 1000 1000
cluster 319 is done ( 90 opinions ) 1000 1000 1000
cluster 634 is done ( 89 opinions ) 1000 1000 1000
cluster 832 is done ( 89 opinions ) 1000 1000 1000
cluster 521 is done ( 88 opinions ) 1000 1000 1000
cluster 800 is done ( 86 opinions ) 1000 1000 1000
cluster 844 is done ( 84 opinions ) 1000 1000 1000
cluster 11 is done ( 83 opinions ) 1000 1000 1000
cluster 424 is done ( 83 